In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import datatable as dt
import numpy as np
# import mlb
import pickle as pkl
from tqdm import tqdm
from itertools import product
import lightgbm as lgb
from fuzzywuzzy import fuzz
import re
import numba as nb
from numba import njit
import xgboost as xgb
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/mlb-player-digital-engagement-forecasting/players.csv
/kaggle/input/mlb-player-digital-engagement-forecasting/example_sample_submission.csv
/kaggle/input/mlb-player-digital-engagement-forecasting/teams.csv
/kaggle/input/mlb-player-digital-engagement-forecasting/seasons.csv
/kaggle/input/mlb-player-digital-engagement-forecasting/example_test.csv
/kaggle/input/mlb-player-digital-engagement-forecasting/train_updated.csv
/kaggle/input/mlb-player-digital-engagement-forecasting/train.csv
/kaggle/input/mlb-player-digital-engagement-forecasting/awards.csv
/kaggle/input/mlb-player-digital-engagement-forecasting/mlb/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/mlb-player-digital-engagement-forecasting/mlb/__init__.py


In [2]:
""" Required modules inatlling """
#!pip install pip --upgrade
# !pip install MLB

' Required modules inatlling '

"""
Sample https://www.kaggle.com/code/brandenkmurray/mlb-predict-final/notebook
""""

** Data read started **

In [3]:
train = pd.read_csv("../input/mlb-player-digital-engagement-forecasting/train_updated.csv")
test = True

In [4]:
if test:
    last_date = train['date'].max()
else:
    last_date = 20210430
print(" last date : {}".format(last_date))

 last date : 20210731


**SAMPLE DATA**

**EDA**

In [5]:
print("\n Shape of data \n")
print("==="*30)
print(train.shape)
print("==="*30)
print("\n NUmber of columns ")
print("==="*30)
print(train.columns)
print("==="*30)
print(train.info())
print("==="*30)


 Shape of data 

(1308, 12)

 NUmber of columns 
Index(['date', 'nextDayPlayerEngagement', 'games', 'rosters',
       'playerBoxScores', 'teamBoxScores', 'transactions', 'standings',
       'awards', 'events', 'playerTwitterFollowers', 'teamTwitterFollowers'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1308 entries, 0 to 1307
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   date                     1308 non-null   int64 
 1   nextDayPlayerEngagement  1308 non-null   object
 2   games                    729 non-null    object
 3   rosters                  1307 non-null   object
 4   playerBoxScores          627 non-null    object
 5   teamBoxScores            627 non-null    object
 6   transactions             1194 non-null   object
 7   standings                623 non-null    object
 8   awards                   309 non-null    object
 9   events               

In [6]:
date_nested_table_t = train[['date', "nextDayPlayerEngagement"]]

**Utility Functions**

In [7]:
def unnest(data, name):
    try:
        date_nested_table = data[['date', name]]

        date_nested_table = (date_nested_table[
          ~pd.isna(date_nested_table[name])
          ].
          reset_index(drop = True)
          )

        daily_dfs_collection = []
        print("\n daily_dfs_collection started ")
        for date_index, date_row in date_nested_table.iterrows():
            daily_df = pd.read_json(date_row[name])

            daily_df['dailyDataDate'] = date_row['date']

            daily_dfs_collection = daily_dfs_collection + [daily_df]

        if daily_dfs_collection:
            # Concatenate all daily dfs into single df for each row
            unnested_table = (pd.concat(daily_dfs_collection,
              ignore_index = True).
              # Set and reset index to move 'dailyDataDate' to front of df
              set_index('dailyDataDate').
              reset_index()
              )
            return reduce_mem_usage(unnested_table, False)
        else:
            return pd.DataFrame()
    except Exception as e:
        print(e)
        print(f'unnest failed for {name}. returning empty dataframe')
        return pd.DataFrame()